In [1]:
# !ls fiverr/sampleData/'Sample Data'/courses/
root_path=r'Sample_Data/'
courses_path=root_path+'courses'

In [2]:
#getting courses files, retrieve data and find course popularity by maintaining course' score

import os
import pandas as pd

no_of_courses=0
courses_names=[]
students=[]
student_enrolled_courses={}

courses_score=pd.DataFrame (columns = ['course_name','score'])

for filename in os.listdir(courses_path):
    courses=os.path.join(courses_path, filename)
    
    no_of_registration=0
    suitability_score=0
    for file in os.listdir(courses):
        file_path=os.path.join(courses, file)
        
        #getting no of registration on courses to get course popularity
        
        if file=='registration.csv':
            registration=pd.read_csv(file_path)
            no_of_registration=registration.shape[0]
            
            #getting unique students for latter use
            course_students=registration['learner_id']
            for student in course_students:
                if student not in students:
                    students.append(student)
                    
                #getting enrolled courses by student for latter use
                if student not in student_enrolled_courses.keys():
                    student_enrolled_courses[student]=[filename]
                else:
                    if filename not in student_enrolled_courses[student]:
                        student_enrolled_courses[student].append(filename)
            
        #getting forum data to find course suitability for recommendation
        
        if file=='forum.csv':
            forum=pd.read_csv(file_path)
            
            for index,row in forum.iterrows():
                lk=row['likes']
                sa=row['SA']
                
                #sentiment analysis factors added as neutral 1, positive 2 and negative -2
                if sa<0:
                    sa+=-1
                else:
                    sa+=1
                    
                #comment with no like is considered 1 like comment to incorporate into analysis
                if lk==0:
                    lk=1
                    
                #score is measured by multiplying likes with sentiment analysis 
                suitability_score+=sa*lk
      
        course_score=(no_of_registration*2)+suitability_score
    courses_score.loc[len(courses_score)]=[filename,course_score]
        
    no_of_courses+=1
    courses_names.append(filename)

print("###### Course score ######")
print(courses_score)
print("\n###### Unique students ######")
print(students)
print("\n###### Courses enrolled by every student ######")
print(student_enrolled_courses)

###### Course score ######
  course_name score
0     Course2    47
1     Course1    75

###### Unique students ######
['nume-552', 'hrbf-553', 'tnwm-944', 'uafi-528', 'jtwn-324', 'pwen-653', 'hsbf-583', 'dpwn-364', 'apen-673', 'some-582', 'aowm-994', 'rsfi-588']

###### Courses enrolled by every student ######
{'nume-552': ['Course2'], 'hrbf-553': ['Course2'], 'tnwm-944': ['Course2'], 'uafi-528': ['Course2'], 'jtwn-324': ['Course2'], 'pwen-653': ['Course2'], 'hsbf-583': ['Course1'], 'dpwn-364': ['Course1'], 'apen-673': ['Course1'], 'some-582': ['Course1'], 'aowm-994': ['Course1'], 'rsfi-588': ['Course1']}


In [3]:
#getting student information and similarity factors

#similarity factors
# 1. Do student comment on forum?
# 2. Do student give quizzes?
# 3. What is student steps' behaviour during course.
# 4. How much student himself/herself similar with other students i.e age,sex,location.

student_info=pd.DataFrame (columns = ['student_id','gender','country','age_group','education','employment'])
forum_info=pd.DataFrame (columns = ['student_id','has_commented'])
quiz_info=pd.DataFrame (columns = ['student_id','has_given_quiz'])
behaviour_info=pd.DataFrame (columns = ['student_id','steps','types'])

#HELPING FUNCTIONS: Calculate Similarity function

from collections import Counter
import math

def calculate_similarity(list1,list2): #dot product similarity function
    c1 = Counter(list1)
    c2 = Counter(list2)
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return round(dotprod / (magA * magB),4)

def check_similarity(row1,row2,field): #custom made similarity function
    if row1[field]==row2[field]:
        return 0.5555
    else:
        return 0.1111
        

# 1. has student commented on forum, 0.9 for those who commented and 0.1 for who didn't comment

for filename in os.listdir(courses_path): #get students who do comments
    courses=os.path.join(courses_path, filename)
    for file in os.listdir(courses):
        file_path=os.path.join(courses, file)
        if file=='forum.csv':
            forum=pd.read_csv(file_path)
            for index,row in forum.iterrows():
                if row['author_id'] not in forum_info['student_id'].tolist():
                    forum_info.loc[len(forum_info)]=[row['author_id'],1]

for student in students: #get students who do not comments
    if student not in forum_info['student_id'].tolist():
        forum_info.loc[len(forum_info)]=[student,0]
            
forum_similarity_score={}
for index,row in forum_info.iterrows(): #get similarity scores among students
    fscore_dict={}
    for index2,row2 in forum_info.iterrows():
        if row2['student_id']!=row['student_id']:
            fscore_dict[row2['student_id']]=check_similarity(row,row2,'has_commented')
    forum_similarity_score[row['student_id']]=fscore_dict

print("####### SIMILARITY OF EVERY STUDENT WITH OTHER STUDENTS IN DIFFERENT PERSPECTIVE #######")
print("\n\n####### FORUM SIMILARITY #######")
print(len(forum_similarity_score),forum_similarity_score)

# 2. has student given quiz, 0.5 for those who gave quiz and 0.1 for who didn't give

for filename in os.listdir(courses_path): #get students who give quizzes
    courses=os.path.join(courses_path, filename)
    for file in os.listdir(courses):
        file_path=os.path.join(courses, file)
        if file=='quizzes.csv':
            quiz=pd.read_csv(file_path)
            for index,row in quiz.iterrows():
                if row['learner_id'] not in quiz_info['student_id'].tolist():
                    quiz_info.loc[len(quiz_info)]=[row['learner_id'],1]

for student in students: #get students who do not give quizzes
    if student not in quiz_info['student_id'].tolist():
        quiz_info.loc[len(quiz_info)]=[student,0]
        
quiz_similarity_score={}
for index,row in quiz_info.iterrows(): #get similarity scores among students
    qscore_dict={}
    for index2,row2 in quiz_info.iterrows():
        if row2['student_id']!=row['student_id']:
            qscore_dict[row2['student_id']]=check_similarity(row,row2,'has_given_quiz')
    quiz_similarity_score[row['student_id']]=qscore_dict

print("\n####### QUIZ SIMILARITY #######")
print(len(quiz_similarity_score),quiz_similarity_score)

# 3. Students steps' behavior during course

students_maintained_steps={}
for filename in os.listdir(courses_path): #get every student steps' behaviour
    courses=os.path.join(courses_path, filename)
    for file in os.listdir(courses):
        file_path=os.path.join(courses, file)
        if file=='step_log.csv':
            step_log=pd.read_csv(file_path)
            for index,row in step_log.iterrows():
                if row['learner_id'] not in students_maintained_steps.keys():
                    students_maintained_steps[row['learner_id']]=[row['step']]
                else:
                    students_maintained_steps[row['learner_id']].append(row['step'])
        if file=='steps.csv':
            steps=pd.read_csv(file_path)
            
for student in students: #get students who did not start course and give them step 1_1(it will be their first step)
    if student not in students_maintained_steps.keys():
        students_maintained_steps[student]=['1_1']
   
students_steps_converted={}
for student in students_maintained_steps: #convert step to their type (i.e wiki,video etc)
    students_steps_converted[student]=[]
    for step in students_maintained_steps[student]:
        step_type=steps.loc[steps['step']==step,'type'].iloc[0]
        students_steps_converted[student].append(step_type)


for stu in students_maintained_steps: #merge step and type of every student into one dataframe for better visualisation
    behaviour_info.loc[len(behaviour_info)]=[stu,students_maintained_steps[stu],students_steps_converted[stu]]

temp_step_sim_score={} #calculate similarity for both steps and types
temp_type_sim_score={}
for index,row in behaviour_info.iterrows():
    step_score_dict={}
    type_score_dict={}
    for index2,row2 in behaviour_info.iterrows():
        if row2['student_id']!=row['student_id']:
            step_score_dict[row2['student_id']]=calculate_similarity(row['steps'],row2['steps'])
            type_score_dict[row2['student_id']]=calculate_similarity(row['types'],row2['types'])
    temp_step_sim_score[row['student_id']]=step_score_dict
    temp_type_sim_score[row['student_id']]=type_score_dict
    
behaviour_similarity_score={}
for student in temp_step_sim_score: #convert step and type similarity score into final similarity score
    total_score={}
    other_students_from_step=temp_step_sim_score[student]
    other_students_from_type=temp_type_sim_score[student]
    for other_student in other_students_from_step:
        score=0.1111
        step_value=other_students_from_step[other_student]
        type_value=other_students_from_type[other_student]
        score=round((1.5*step_value)+type_value,4) #step is given more priority (1.5) as compared to type(1)
        total_score[other_student]=score
    behaviour_similarity_score[student]=total_score #final behaviour similarity score
   
print("\n####### BEHAVIOUR SIMILARITY #######")
print(len(behaviour_similarity_score),behaviour_similarity_score)

#4. student himself/herself similarity with other students

for filename in os.listdir(courses_path): #get students information
    courses=os.path.join(courses_path, filename)
    for file in os.listdir(courses):
        file_path=os.path.join(courses, file)
        if file=='registration.csv':
            registration=pd.read_csv(file_path)
            
            for index,row in registration.iterrows():
                if row['learner_id'] not in student_info['student_id'].tolist():
                    lid=row['learner_id']
                    gen=row['gender']
                    con=row['country']
                    age=row['age_group']
                    edu=row['education']
                    emp=row['employment']
                    student_info.loc[len(student_info)]=[lid,gen,con,age,edu,emp]

student_similarity_score={}
for index,row in student_info.iterrows(): #get similarity among students
    score_dict={}
    for index2,row2 in student_info.iterrows():
        if row2['student_id']!=row['student_id']:
            score_dict[row2['student_id']]=calculate_similarity(row,row2)
    student_similarity_score[row['student_id']]=score_dict
  
print("\n####### STUDENT SIMILARITY #######")
print(len(student_similarity_score),student_similarity_score)

####### SIMILARITY OF EVERY STUDENT WITH OTHER STUDENTS IN DIFFERENT PERSPECTIVE #######


####### FORUM SIMILARITY #######
12 {'nume-552': {'pwen-653': 0.5555, 'jtwn-324': 0.5555, 'hsbf-583': 0.5555, 'apen-673': 0.5555, 'some-582': 0.5555, 'hrbf-553': 0.1111, 'tnwm-944': 0.1111, 'uafi-528': 0.1111, 'dpwn-364': 0.1111, 'aowm-994': 0.1111, 'rsfi-588': 0.1111}, 'pwen-653': {'nume-552': 0.5555, 'jtwn-324': 0.5555, 'hsbf-583': 0.5555, 'apen-673': 0.5555, 'some-582': 0.5555, 'hrbf-553': 0.1111, 'tnwm-944': 0.1111, 'uafi-528': 0.1111, 'dpwn-364': 0.1111, 'aowm-994': 0.1111, 'rsfi-588': 0.1111}, 'jtwn-324': {'nume-552': 0.5555, 'pwen-653': 0.5555, 'hsbf-583': 0.5555, 'apen-673': 0.5555, 'some-582': 0.5555, 'hrbf-553': 0.1111, 'tnwm-944': 0.1111, 'uafi-528': 0.1111, 'dpwn-364': 0.1111, 'aowm-994': 0.1111, 'rsfi-588': 0.1111}, 'hsbf-583': {'nume-552': 0.5555, 'pwen-653': 0.5555, 'jtwn-324': 0.5555, 'apen-673': 0.5555, 'some-582': 0.5555, 'hrbf-553': 0.1111, 'tnwm-944': 0.1111, 'uafi-528': 0.111


####### STUDENT SIMILARITY #######
12 {'nume-552': {'hrbf-553': 0.8889, 'tnwm-944': 0.6667, 'uafi-528': 0.6667, 'jtwn-324': 0.8165, 'pwen-653': 0.0, 'hsbf-583': 0.0, 'dpwn-364': 0.0, 'apen-673': 0.9245, 'some-582': 0.9245, 'aowm-994': 0.0962, 'rsfi-588': 0.9245}, 'hrbf-553': {'nume-552': 0.8889, 'tnwm-944': 0.6667, 'uafi-528': 0.6667, 'jtwn-324': 0.8165, 'pwen-653': 0.0, 'hsbf-583': 0.0, 'dpwn-364': 0.0, 'apen-673': 0.9245, 'some-582': 0.9245, 'aowm-994': 0.0962, 'rsfi-588': 0.9245}, 'tnwm-944': {'nume-552': 0.6667, 'hrbf-553': 0.6667, 'uafi-528': 0.5, 'jtwn-324': 0.6124, 'pwen-653': 0.2887, 'hsbf-583': 0.433, 'dpwn-364': 0.0, 'apen-673': 0.6934, 'some-582': 0.6934, 'aowm-994': 0.2887, 'rsfi-588': 0.6934}, 'uafi-528': {'nume-552': 0.6667, 'hrbf-553': 0.6667, 'tnwm-944': 0.5, 'jtwn-324': 0.6124, 'pwen-653': 0.0, 'hsbf-583': 0.0, 'dpwn-364': 0.433, 'apen-673': 0.6934, 'some-582': 0.6934, 'aowm-994': 0.0, 'rsfi-588': 0.6934}, 'jtwn-324': {'nume-552': 0.8165, 'hrbf-553': 0.8165, 'tnwm-944

In [4]:
#Now add all the similarity scores to find the total similarity score

final_similarity_score={}
for student in forum_similarity_score: #get all similarity of a student one by one
    forum_score=forum_similarity_score[student]
    quiz_score=quiz_similarity_score[student]
    behaviour_score=behaviour_similarity_score[student]
    student_score=student_similarity_score[student]
    temp_student_sim_score={}
    for other_student in forum_score:
        f_score=forum_score[other_student]
        q_score=quiz_score[other_student]
        b_score=behaviour_score[other_student]
        s_score=student_score[other_student]
        final_score=(1.5*b_score)+(1.2*s_score)+(1.0*q_score)+(1.0*f_score) #behaviour and student similarity score is given more priority
        temp_student_sim_score[other_student]=round(final_score,4)
    final_similarity_score[student]=temp_student_sim_score
    
# print(final_similarity_score)

#get the most similar users, in order, for every user

most_similar_student={}
for student in final_similarity_score:
    other_student=final_similarity_score[student]
    sorted_other_student=sorted(other_student.items(), key = lambda kv:(kv[1], kv[0]),reverse=True) #sort in descending order
    temp_list=[]
    for std in sorted_other_student: #prepare list of ordered most similar student
        temp_list.append(std[0])
    most_similar_student[student]=temp_list 

# print(most_similar_student)

#get the suggested course

suggested_course={}
no_of_course_suggestion=1 #How many course you want to suggest

for student in most_similar_student:
    similar_students=most_similar_student[student] #get most similar students of a student
    course_list=[]
    for stu in similar_students:
        courses_enrolled_by_stu=student_enrolled_courses[stu] #get course enrolled by similar student (go upward for student_enrolled_courses)
        for course in courses_enrolled_by_stu:
            if course not in student_enrolled_courses[student]: #check if course is not already taken by this student
                course_list.append(course)
                if len(course_list)==no_of_course_suggestion: #check no of course suggesting
                    break
        if len(course_list)==no_of_course_suggestion:
            break
    suggested_course[student]=course_list #prepare suggesting courses

print("###### COURSE SUGGESTION FOR EVERY STUDENT ######\n")
# print(suggested_course)
for student in suggested_course:
    print("Course suggested to",student,": ")
    for suggestion in suggested_course[student]:
        print(suggestion,",")

###### COURSE SUGGESTION FOR EVERY STUDENT ######

Course suggested to nume-552 : 
Course1 ,
Course suggested to pwen-653 : 
Course1 ,
Course suggested to jtwn-324 : 
Course1 ,
Course suggested to hsbf-583 : 
Course2 ,
Course suggested to apen-673 : 
Course2 ,
Course suggested to some-582 : 
Course2 ,
Course suggested to hrbf-553 : 
Course1 ,
Course suggested to tnwm-944 : 
Course1 ,
Course suggested to uafi-528 : 
Course1 ,
Course suggested to dpwn-364 : 
Course2 ,
Course suggested to aowm-994 : 
Course2 ,
Course suggested to rsfi-588 : 
Course2 ,


SyntaxError: invalid syntax (<ipython-input-6-5a7b4446f16d>, line 1)